# Task Scheduling with Swarm Intelligence 
an Introduction with case : Penjadwalan Sidang Mahasiswa Tingkat Akhirat

___

In [1]:
import pandas as pd
import numpy as np
import math
from natsort import natsorted
import warnings
warnings.filterwarnings("ignore")

In [2]:
mhs = pd.read_csv('data/mahasiswa.csv')
dosen = pd.read_csv('data/dosen.csv')
ruang = pd.read_csv('data/ruang.csv')
jadwal_dosen = pd.read_csv('data/jadwal_dosen.csv')

### Problem Statement

Task scheduling with constraints categorized into NP problem wich stands for Non Deterministic Polinomial that takes quick time to check whether the solution is correct or not, but takes long time to solve.

![](assets/np_hardness.png)

In this case we are having **124** students that need to be scheduled in **13** specific room at **20** available time slot. Each student should be acompanied with **at least one** teacher (max: 2). 

In [3]:
mhs

,nama,nim,dp1,dp2,judul_ta
0,mhs_0,nim_0,dosen_69,dosen_61,judul_0
1,mhs_1,nim_1,dosen_20,dosen_20,judul_1
2,mhs_2,nim_2,dosen_19,dosen_2,judul_2
3,mhs_3,nim_3,dosen_44,dosen_44,judul_3
4,mhs_4,nim_4,dosen_4,dosen_2,judul_4
...,...,...,...,...,...
119,mhs_119,nim_119,dosen_43,dosen_48,judul_119
120,mhs_120,nim_120,dosen_33,dosen_45,judul_120
121,mhs_121,nim_121,dosen_44,dosen_36,judul_121
122,mhs_122,nim_122,dosen_33,dosen_38,judul_122


In [4]:
ruang

,ruang,kk
0,Ruang Rapat Gedung D lt 1,ALL
1,Ruang Rapat ICM,ICM
2,Lab UC,TELE
3,Lab HES,TELE
4,Lab NOS,TELE
5,Lab Foresty,TELE
6,Lab DMC,"SIDE, MCE"
7,Lab RPL,"SIDE, MCE"
8,Lab Tel-C,"SIDE, MCE"
9,Lab Basis Data,"SIDE, MCE"


### Problem Complexity
Menjadwalkan : 
- m = 1 mahasiswa (kemunginan : 124)
- s = pada 1 slot tempat x waktu (kemngkinan: 13 * 20)
- d = 1 atau 2 dosen untuk mendampingi (kemungkinan : 3)

Complexity : $$ C_{m}^s \times d$$

Order of Complexity : Combinatorial

In [6]:
math.comb((13*20), 124) * 3

208501360578020938029361385623653699574688684968360947127928832021966924612000

In [7]:
from natsort import natsorted
jdw = jadwal_dosen.pivot_table(
    index='dosen', 
    columns=['day', 'time'], 
    values='val',
    aggfunc='sum'
)[['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']]

jdw = jdw.reindex(natsorted(jdw.index))
# jdw

In [8]:
jdw.columns = jdw.columns.map('_'.join)
# jdw

# How to Solve the Problem 
1. Meta-Heuristic Searching : Swarm Intelligence
1. Reinforcement Learning


___

## Swarm Intelligence

![](assets/ai.png)

**TLDR : Swarm intelligence is Optimization Technique insipired by Swarm Behaviour in Nature**

Some reference on Swarm Intelligence Algorithms: 
- [Ant Colony Optimization](https://en.wikipedia.org/wiki/Ant_colony_optimization_algorithms)
- [Bat Algorithm](https://www.researchgate.net/publication/255971823_Bat_Algorithm_Literature_Review_and_Applications)
- [Bee Colony Optimization](https://www.researchgate.net/publication/226530817_Bee_colony_optimization_BCO)
- [Cat Swarm Optimization](https://www.researchgate.net/publication/221419703_Cat_Swarm_Optimization)
- [Cuckoo Algorithm](https://www.sciencedirect.com/science/article/pii/S2210832717301679)
- [Dolphin Swarm Algorithm](https://link.springer.com/article/10.1631/FITEE.1500287)
- [Firefly Algorithm](https://www.sciencedirect.com/topics/engineering/firefly-algorithm)
- [Particle Swarm Optimization](https://www.sciencedirect.com/topics/engineering/particle-swarm-optimization)


## 1. State Definition 
Model a problem into measurable state

In [9]:
state = mhs[['nama']]
state['dp1'] = 0
state['dp2'] = 0 
state['ruang'] = np.random.choice(ruang.ruang, size=len(mhs))
# state['time'] = np.random.choice(jadwal_dosen['time'].unique(),  size=len(mhs))
state['slot'] = np.random.randint(0,20, size=len(mhs))

In [10]:
def random_sol(to_df=False):
    sol = np.zeros(shape=(len(mhs),4))
    sol[:,0] = np.random.randint(0,2, size=len(mhs)) #dp1
    sol[:,1] = np.random.randint(0,2, size=len(mhs)) #dp2
    sol[:,2] = np.random.randint(0,len(ruang), size=len(mhs)) #ruang
    sol[:,3] = np.random.randint(0,20, size=len(mhs))
    
    if to_df :
        df = pd.DataFrame(sol, columns=['dp1', 'dp2', 'ruang', 'slot'])
        return df
    else: return sol
    
ub = [1,1,len(ruang)-1, 19]
lb = [0,0,0,0]

In [11]:
state

,nama,dp1,dp2,ruang,slot
0,mhs_0,0,0,Lab NOS,5
1,mhs_1,0,0,Lab RPL,7
2,mhs_2,0,0,Lab NOS,1
3,mhs_3,0,0,Lab Tel-C,11
4,mhs_4,0,0,Lab Tel-C,15
...,...,...,...,...,...
119,mhs_119,0,0,Lab Multimedia,1
120,mhs_120,0,0,Lab HES,18
121,mhs_121,0,0,Lab Basis Data,1
122,mhs_122,0,0,Lab RPL,17


solutions state would be like : 


In [12]:
no_to_ruang = ruang['ruang'].to_dict()
ruang_to_no = { no_to_ruang[k]:k for k in no_to_ruang}

no_to_slot = pd.Series(jdw.columns).to_dict()
# slot_to_no = {no_to_jam[k]:k for k in no_to_jam}

solstate = random_sol(to_df=True)

In [13]:
solstate

,dp1,dp2,ruang,slot
0,1.0,1.0,11.0,11.0
1,1.0,0.0,9.0,5.0
2,0.0,0.0,11.0,15.0
3,0.0,1.0,4.0,8.0
4,1.0,0.0,7.0,17.0
...,...,...,...,...
119,0.0,1.0,8.0,17.0
120,0.0,1.0,11.0,17.0
121,0.0,0.0,12.0,11.0
122,1.0,0.0,6.0,18.0


In [14]:
a = solstate
b = solstate

## 2. Loss/Fit Function:
- Jumlah dosen yang hadir  = a (Okupansi)
- Jumlah bentrok jadwal dosen = b (Pelanggaran Constraint)
- Kesesuaian kk ruang dan dosen = c (Pelanggaran Constraint)
- durasi jadwal = d
- Jumlah slot bentrok = e

Minimalkan nilai e :

$$ e = \frac{\beta \cdot b + \delta \cdot d + \epsilon \cdot e}{\alpha \cdot a+ \gamma \cdot c }$$

with 
- $\alpha = 1$
- $\beta = 1$
- $\gamma = 1$
- $\delta = 1$
- $\epsilon = 1$



In [15]:
def interpret(x):
    _x = x.round().astype(int)
    return np.clip(_x, lb, ub)

In [16]:
def to_dataframe(x):
    sol = pd.DataFrame(interpret(x))
#     sol[2] = sol[2].replace(no_to_ruang)
    sol.columns=['dp1', 'dp2', 'ruang', 'slot']
    return sol

In [17]:
def is_ngajar(dosen, slot, jadwal):
    return jadwal.loc[dosen].iloc[slot]

In [18]:
def cek_bentrok(df):
    mhs_dosen = mhs.join(df, rsuffix='_target')
    bentrok_dp1 = [is_ngajar(x['dp1'], x['slot'], jdw) for _, x in mhs_dosen.iterrows()] * mhs_dosen['dp1_target']
    bentrok_dp2 = [is_ngajar(x['dp2'], x['slot'], jdw) for _, x in mhs_dosen.iterrows()] * mhs_dosen['dp2_target']
    return bentrok_dp1.sum(), bentrok_dp2.sum()

In [19]:
def fitness(x):
    alpha = 2
    beta = 1
    gamma = 1
    delta = 1 
    epsilon = 2
    df = to_dataframe(x)
    
    n_dosen_hadir = df[['dp1', 'dp2']].sum().sum()
    n_bentrok_dosen_1, n_bentrok_dosen_2 = cek_bentrok(df)
    n_bentrok_dosen = n_bentrok_dosen_1 + n_bentrok_dosen_2
    n_notmatch = 0  # not yet implemented 
    durasi_jadwal = df['slot'].max() // 4
    
    n_slot_bentrok = (df.groupby(['ruang', 'slot']).count() - 1).sum().sum()
    
    return (beta*n_bentrok_dosen + delta*durasi_jadwal  + epsilon*n_slot_bentrok) / (alpha*n_dosen_hadir +  gamma*n_notmatch)

## 3. Performance Metric 
- Berapa banyak Mahasiswa yang bisa sidang (okupansi)
- Berapa banyak dosen yang melakukan sidang (okupansi)

____


# Particle Swarm Optimization Algorithm

![](assets/pso.gif)

1. Generate random agent/solution
2. Measure fitness of each agents
3. Set all agents to move closer to best agent

In [20]:
n_population = 100
learning_rate = 0.05
epoch = 10

In [21]:
# 1. Genrate random solution
agents = [random_sol() for i in range(n_population)]
best_agent_iter = []
best_fit_iter = []

In [22]:
for i in range(epoch):
    # 2. Fitness measurement
    fits = [fitness(x) for x in agents]
    best_agent = agents[np.argmin(fits)]
    best_agent_iter.append(best_agent)
    best_fit_iter.append(fits[np.argmin(fits)])
    
    # set all agents to move closer to the best agent
    agents = [x+((x-best_agent)*learning_rate) for x in agents]
    
    # add random motion to best_agent
    random_agent = agents[np.random.choice(len(agents))]
    agents[np.argmin(fits)] += (random_agent - best_agent) * learning_rate / 10
    
    print(f"epoch: {i} : best fit = {np.min(fits)}")

epoch: 0 : best fit = 0.42857142857142855
epoch: 1 : best fit = 0.42857142857142855
epoch: 2 : best fit = 0.42857142857142855
epoch: 3 : best fit = 0.42857142857142855
epoch: 4 : best fit = 0.42857142857142855
epoch: 5 : best fit = 0.42857142857142855
epoch: 6 : best fit = 0.42857142857142855
epoch: 7 : best fit = 0.42857142857142855
epoch: 8 : best fit = 0.44841269841269843
epoch: 9 : best fit = 0.4444444444444444


___

Explain the output

In [23]:
final_jadwal = to_dataframe(best_agent_iter[np.argmin(best_fit_iter)])

In [24]:
final_jadwal

,dp1,dp2,ruang,slot
0,1,1,7,3
1,0,1,5,19
2,1,1,8,6
3,0,1,10,4
4,0,0,8,9
...,...,...,...,...
119,0,1,1,10
120,0,1,8,11
121,1,1,7,11
122,0,1,2,8
